# Récupération de tweets et d'utilisateurs à l'aide de l'API twitter #
Ce notebook a pour objectif de récupérer des tweets à l'aide de l'API Twitter.
Pour l'utiliser, assurez vous de détenir un accès "elevated" à l'API Twitter : https://developer.twitter.com/en

## Librairies utilisées ##

In [ ]:
!pip install twitter==2.0a2 #Cette version est la plus récente mais n'est pas celle qui apparaît sur PyPi
import pandas as pd
import twitter
from time import time, sleep
import pandas as pd
import json

## Explication du fonctionnement de l'API Twitter ##

Toutes les notions évoquées ici sont détaillées dans la documentation de l'API : https://developer.twitter.com/en/docs

Pour comprendre comment utiliser l'API de Twitter, deux notions sont importantes : \
    -L'Authentification, \
    -Les Endpoints
\
\
Pour effectuer des requêtes sur l'API Twitter, il est nécessaire de disposer des bonnes accréditation et donc de s'authentifier pour en attester. On peut s'authentifier en tant qu'utilisateurs (pour envoyer des tweets automatiquement avec l'API par exemple) ou en tant qu'application. L'authentification se fait à l'aide du protocole __[OAUTH](https://oauth.net/2/)__ et la version 2.0 nécessite un identifiant appelé **bearer** qui est propre à chaque application Twitter.
\
\
Après authentification, on effectue une requête en accédant à un __[**Endpoint**](https://developer.twitter.com/en/docs/api-reference-index)__. Chaque endpoint de l'API correspond à un type de requête particulier. L'accès à un endpoint se fait par URL mais l'intérêt de la librairie Twitter précédemment installé est de faciliter l'authentification.
\
\
L'API Twitter dispose depuis 2018 de deux versions différentes (2.0 et 1.1) dont l'arborescence et le format des réponses aux requêtes sont différents. Pour des raisons pratiques, il arrivera que nous utiliserons plutôt l'une que l'autre, d'où la nécessité de disposer d'un accès "elevated" à l'API (la version 1.1 n'est disponible que pour cet accès là). 

In [ ]:
#Initialisation de l'API
bearer = '' #Rentrez votre clé bearer ici
from twitter import OAuth2
auth = OAuth2(bearer_token=bearer)
api = twitter.Twitter(auth=auth,api_version="2",format="") #On va tout d'abord utiliser la version 2.0 de l'API

## L'utilisation des requêtes ##


Dans la plupart des cas, les requêtes renvoient des dictionnaires au format json avec une clé "data" qui contient les informations principales, une clé "includes" qui contient les informations optionnelles au cas où des paramètres supplémentaires ont été ajoutés, et une clé "meta" qui contient des informations annexes.

### La recherche de tweets ###

On cherche ici à analyser les communautés qui se forment autour d'un sujet sur Twitter, ce peut être un hashtag ou des mots clés. On va pour cela utiliser l'endpoint de __[recherche de tweets récents](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent)__ 

In [ ]:
#La fonction suivante traduit le dictionnaire renvoyé par la recherche de tweets via l'API en DataFrame pandas
def req_to_df(req) :
    """
    Entrée : req un objet de type TwitterDictResponse de la libraire twitter venant d'une recherche de tweets
    Sortie : 
        df_tweets : un dataframe contenant des informations sur les tweets
        df_users : un dataframe contenant des informations sur les utilisateurs
    """
    df_tweets = pd.DataFrame(req['data']) #DataFrame des tweets
    df_users = pd.DataFrame(req['includes']['users']) #DataFrame des utilisateurs
    return(df_tweets,df_users)

La recherche de tweets via l'API fait appel à des options dont notament des __[**query**](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)__ qui sont des filtres de recherche portant sur le texte des tweets (c'est l'équivalent de la barre de recherche sur Twitter). Les querys ne peuvent pas faire plus de 512 caractères. D'autres options portant sur la date de post du tweet ou les informations sur l'utilisateurs qui a posté le tweet peuvent aussi être ajoutées.

### La fonction de recherche de tweets ### 
La fonction suivante permet de chercher des tweets selon une certaine query et entre deux dates précises. L'endpoint a une capacité d'utilisation de 450 requêtes toutes les 15 minutes et pour racourcir le temps de récupération ainsi que les analyses futures, seules 400 seront utilisées dans cette fonction. Cela représente donc 40 000 tweets récupérables.\
Cette fonction renvoie un DataFrame comportant l'id, l'id de l'auteur, la date de création, le texte ainsi que le nombre de likes et de retweets des tweets recherchés. Ainsi qu'un DataFrame comportant l'id, le nom apparent, le nom d'utilisateur, le nombre de followers et de followings des utilisateurs qui ont tweetés selon la requête entrée.\
La fonction sauvegarde aussi automatiquement les DataFrame au format json.

In [ ]:
def query_to_df(query,deb,fin,twitter_api,save_t,save_u) :
    """
    Entrée :
        query : un string contenant la query de la recherche pour plus d'info https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query
        deb : la date de début de la recherche au format 'AAAA-MM-JJTHH:MM:SSZ'
        fin : la date de fin de la recherche au format 'AAAA-MM-JJTHH:MM:SSZ'
        twitter_api : un objet de type API de la libraire twitter
        save_t : nom du fichier de sauvegarde des tweets
        save_u : nom du fichier de sauvegarde des utilisateurs
    Sortie :
        df_tweets : un dataframe des tweets (hors retweets) postés entre la date de début et de fin de la recherche selon la query indiquée
        df_users :  un dataframe des utilisateurs ayant posté des tweets entre la date de début et de fin de la recherche selon la query indiquée
    """
    request = twitter_api.tweets.search.recent(
        query=query,
        start_time = deb,
        end_time = fin,
        max_results = 100,
        expansions = 'author_id',
        params = {
            'tweet.fields' : 'created_at,public_metrics',
            'user.fields' : 'id,name,description,public_metrics'
        }
    )
    df_tweets,df_users = req_to_df(request)
    oldest = request['meta']['oldest_id']

    #L'utilisation de l'option until_id et since_id est incompatible avec l'utilisation de start_time et end_time, on va donc délimiter les tweets entre deux id
    request_old = twitter_api.tweets.search.recent(
        query=query,
        end_time = deb,
        max_results = 100,
    ) 
    fst = request_old['meta']['newest_id']

    for k in range(400) : #400*100 = 40 000 tweets scrappés au max
        request = twitter_api.tweets.search.recent(
            query=query,
            max_results = 100,
            since_id = fst,
            until_id = oldest,
            expansions = 'author_id',
            params = {
                'tweet.fields' : 'created_at,public_metrics',
                'user.fields' : 'id,name,description,public_metrics'
                }
        )
        if request['meta']['result_count'] == 0 :
            break
        df_t,df_u = req_to_df(request)
        df_tweets = pd.concat([df_tweets,df_t],ignore_index=True)
        df_users = pd.concat([df_users,df_u],ignore_index= True)
        oldest = request['meta']['oldest_id']
        
    df_users = df_users.drop_duplicates(subset = 'id')
    df_tweets.to_json(save_t + ".json") #Sauvegarde des bases dans un fichier json
    df_users.to_json(save_u + ".json")
    return(df_tweets,df_users)

In [ ]:
start = 'AAAA-MM-JJTHH:MM:SS:Z' #La date de début de recherche
stop = 'AAAA-MM-JJTHH:MM:SS:Z' #La date de fin de recherche
query = '# -is:retweet lang:fr' #Les paramètres -is:retweet et lang:fr permettent de ne garder que les tweets en français qui ne sont pas des retweets
name_t = '' #Nom du fichier d'enregistrement du DataFrame tweets
name_u = '' #Nom du fichier d'enregistrement du DataFrame users
df1,df2 = query_to_df(query=query,deb = start,fin = stop,twitter_api=api,save_t=name_t,save_u=name_u)

## La recherche d'utilisateurs ##

Afin de créer un graphe entre des utilisateurs twitter, il faut pouvoir relier chaque utilisateur entre eux selon si un utilisateur suit un autre ou non. Pour faciliter la récupération et les analyses, les graphes ne seront pas orientés, il y a donc équivalence entre l'utilisateur A suit l'utilisateur B et inversement.\
On peut donc se permettre de regarder seulement la liste des followings d'un utilisateur pour voir à quel autre utilisateur il est relié. Cela facilite grandement les analyses. En effet, imaginons que le compte d'Emmanuel Macron soit présent dans les noeuds du graphes, il faudrait alors regarder ses plus de 9 millions de followers ce qui prendrait bien trop de temps.

### Comment tricher avec l'API Twitter ###

Même si regarder la liste des followings est optimisées, les endpoints de récupération des followings ne permet que 15 requêtes par tranches de 15 minutes par applications. Notre base de test comprenait environ 5700 utilisateurs, ce qui aurait pris 95 heures avec une seule application, en supposant qu'une requête suffirait à chaque utilisateur.\
\
Une solution aurait été de créer un grand nombre de compte twitter et ainsi avoir plusieurs applications. Heureusement, avec un accès "elevated", il est possible de créer jusqu'à 9 *Standalone Apps* qui ont accès à l'API 1.1 mais qui des capacités de requêtes indépendantes. Nous avons donc utilisé 20 applications pour récupérer les followings ce qui a réduit le temps de récupération à environ 5h.\
\
Il est donc primordial de disposer de comptes développeur qui bénéficient de l'accès "elevated".

La fonction suivante va donc créer, à partir d'un dictionnaire de *bearers* une liste d'APIs qui seront utilisées pour la récupération des followings.

In [ ]:
def bearers_to_apis(bearers,version) :
    """
    Entrée :
        bearers : une liste de bearers dont la structure est identique à celle de bearers.json
        version : la version de l'api twitter voulue (au format string)
    Sortie :
        apis_list : une liste d'api du module twitter
    """
    apis_list = []
    for bear in bearers : 
        b_auth = OAuth2(bearer_token= bear['bearer'])
        b_to_api = twitter.Twitter(auth=b_auth,api_version=version)
        apis_list.append(b_to_api)
    return(apis_list)

### La fonction de récupération des followings ###

On va alors utiliser l'endpoint de __[récupération des followings](https://developer.twitter.com/en/docs/twitter-api/v1/accounts-and-users/follow-search-get-users/api-reference/get-friends-ids)__ de l'API 1.1. Chaque requête permet de récupérer jusqu'à 5000 followings d'un utilisateurs. Si tout les followings ne peuvent être affichés, des tokens de naviguations permettent de naviguer dans les différentes pages de followings. La fonction suivante va donc répurérer pour chaque utilisateurs d'une liste prédéfinie la liste des comptes qu'il suit. La fonction permet de changer d'applications lorsque le nombre de requêtes maximal est atteint. Elle permet aussi de sauvegarder automatiquement la liste au fur et à mesure de sa construction.

In [ ]:
def following_scrapper(ids_list,following_number,apis_list,save) :
    """
    Entrée :
        ids_list : la liste des ids des utilisateurs twitter dont il faut récupérer la liste des comptes following (ids au format string)
        following_number : la liste du nombre de following d'un utilisateur selon sont indice dans ids_list
        apis_list : la liste des apis de la librairie twitter, de version 1.1 dont on va se servir pour récupérer la liste des following
        save : nom du fichier de sauvegarde, au format str
    Sortie : 
        dict_following : un dictionnaire où chaque clé est l'id d'un utilisateur twitter et qui renvoie la liste de ses following 
    """
    n = len(apis_list) #nombre total d'apis disponibles
    i = 0 #id de l'api utilisée
    dict_following = dict()
    for k in range(len(ids_list)) :
        user = ids_list[k] #id de l'utilisateur dont on récupère les followings
        if following_number[k] == 0 : #il n'y a tout simplement pas de following à récupérer
            dict_following[user] = []
            continue
        following = [] #liste des following de l'utilisateur actuel
        page = '-1' #page de la liste des following que l'on regarde, initialement '-1'
        while page != '0' : #la page "vide" dont le numéro est 0 est celle qui conclue la liste des following
            try :
                f_request = apis_list[i].friends.ids(user_id = user,stringify_ids = True,cursor = page,count = 5000)
                page = f_request['next_cursor_str'] #Changement de la page
                following += f_request['ids'] #Ajout des followings ajoutés
                if f_request.rate_limit_remaining == 0 : #Il n'y a plus de requêtes disponibles pour l'API
                    print('Plus de requêtes disponibles, changement de l\'API utilisée ['+str(i+1)+'/'+str(n)+']')
                    i = (i+1)%n
                    if i == 0 : #si au moment de changer d'api, on revient à l'api initiale, on attend le temps qu'il faut pour réinitialiser le nombre de requêtes disponibles
                        print('La première API n\'a pas eu le temps d\'être réinitialisée, veuillez attendre 15 minutes')
                        sleep(15*60) #les fenêtres d'utilisation des requêtes sont de 15 minutes
            except Exception as e : #Dans le cas d'une erreur
                if e.e.code == 401 : #Cette erreur arrive souvent dans le cas où l'utilisateur recherché a été suspendu
                    print('Le compte de cet utilisateur a été suspendu')
                    break
                if e.e.code == 404 : #Cette erreur arrive souvent dans le cas où l'utilisateur n'existe plus
                    print('Le compte a été supprimé')
                    break
                #Dans le doute on change d'api
                print('Il y a eu une erreur dans l\'utilisation de l\'API, changement de l\'API ['+str(i+1)+'/'+str(n)+']' + ' ('+ str(e)+')')
                i = (i+1)%n
                if i == 0 : #si au moment de changer d'api, on revient à l'api initiale, on attend le temps qu'il faut pour réinitialiser le nombre de requêtes disponibles 
                    print('La première API n\'a pas eu le temps d\'être réinitialisée, veuillez attendre 15 minutes')
                    sleep(15*60) #les fenêtres d'utilisation des requêtes sont de 15 minutes
                continue
        dict_following[user] = following #Mise à jour du dictionnaire
        print('Avancement : '+str(k+1)+'/'+str(len(ids_list))+' terminé')
        if k%100 == 0 : #On sauvegarde tout les 100 utilisateurs
            with open(save + '.json', 'w') as fp:
                json.dump(dict_following, fp)
    with open(save+'.json', 'w') as fp: #enregistrement final
                json.dump(dict_following, fp)
    return(dict_following)


In [ ]:
#Chargement de toutes les APIs utilisées
f = open('bearers.json') 
data = json.load(f)
f.close()
apis = bearers_to_apis(bearers = data,version = "1.1")

#Chargement de la base utilisateur
f = open('.json') #Le dictionnaire de stockage est supposé avoir été construit à l'aide des fonctions de récupérations définies précédemment
users = json.load(f)
f.close()

#Définition de la liste des utilisateurs à récupérer ainsi que leur nombre de followers
ids_users = [users['id'][str(k)] for k in list(users['id'].keys())]
nb_following = [users['public_metrics'][str(k)]['following_count'] for k in list(users['id'].keys())]

In [ ]:
name_save = '' #Nom du fichier de sauvegarde

followings = following_scrapper(ids_list=ids_users,following_number=nb_following,apis_list=apis,save = name_save)